In [1]:
import csv
import sqlite3
import functions
import pandas as pd
import requests as rq
from config import api_key
# pip install geopy
import time
from math import ceil
import pickle

In [2]:
df = pickle.load(open('df_2.pkl','rb'))

In [3]:
df

,Airline,Month Flown,Year Flown,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended,...,Title Polarity,Title Subjectivity,Class_Business Class,Class_Economy Class,Class_First Class,Class_Premium Economy,Type of Traveller_Business,Type of Traveller_Couple Leisure,Type of Traveller_Family Leisure,Type of Traveller_Solo Leisure
0,Singapore Airlines,12,2023,4,4,4,4,4,9,1,...,0.600,0.9,True,False,False,False,False,False,False,True
1,Singapore Airlines,2,2024,5,3,4,4,1,3,0,...,-1.000,1.0,False,True,False,False,False,False,False,True
2,Singapore Airlines,2,2024,1,5,2,1,5,10,1,...,0.000,0.0,False,True,False,False,False,False,True,False
3,Singapore Airlines,2,2024,5,5,5,5,5,10,1,...,0.300,0.3,False,True,False,False,False,False,False,True
4,Singapore Airlines,2,2024,5,5,5,5,5,10,1,...,0.700,0.6,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8095,Korean Air,6,2016,5,4,5,3,4,7,1,...,0.000,0.0,False,True,False,False,True,False,False,False
8096,Korean Air,6,2016,3,5,5,4,5,10,1,...,0.000,0.0,False,True,False,False,False,True,False,False
8097,Korean Air,4,2016,4,5,5,5,1,2,0,...,0.000,0.0,True,False,False,False,True,False,False,False
8098,Korean Air,4,2016,5,1,3,4,5,10,1,...,0.200,0.1,True,False,False,False,True,False,False,False


In [4]:
print(f'Rows and Columns: {df.shape}')
df.columns

Rows and Columns: (5407, 31)


Index(['Airline', 'Month Flown', 'Year Flown', 'Seat Comfort', 'Staff Service',
       'Food & Beverages', 'Inflight Entertainment', 'Value For Money',
       'Overall Rating', 'Recommended', 'Origin', 'Destination', 'Via',
       'Origin_Code', 'Destination_Code', 'Via_Code', 'Origin_City',
       'Destination_City', 'Via_City', 'Comment Polarity',
       'Comment Subjectivity', 'Title Polarity', 'Title Subjectivity',
       'Class_Business Class', 'Class_Economy Class', 'Class_First Class',
       'Class_Premium Economy', 'Type of Traveller_Business',
       'Type of Traveller_Couple Leisure', 'Type of Traveller_Family Leisure',
       'Type of Traveller_Solo Leisure'],
      dtype='object')

In [5]:
airport_positions_df = pd.read_csv('Resources/airport_positions.csv')
# airports_utf_df = pd.read_csv('Resources/airports_utf.csv')

In [6]:
df = df.merge(airport_positions_df, how='left', left_on='Origin_Code', right_on='Airport Code')
df = df.rename(columns={'Latitude': 'Origin_Lat', 'Longitude': 'Origin_Lon'})

In [7]:
df = df.merge(airport_positions_df, how='left', left_on='Destination_Code', right_on='Airport Code')
df = df.rename(columns={'Latitude': 'Dest_Lat', 'Longitude': 'Dest_Lon'})

In [8]:
df = df.drop(['Airport Code_x', 'Airport Code_y'], axis=1)
df

,Airline,Month Flown,Year Flown,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended,...,Class_First Class,Class_Premium Economy,Type of Traveller_Business,Type of Traveller_Couple Leisure,Type of Traveller_Family Leisure,Type of Traveller_Solo Leisure,Origin_Lat,Origin_Lon,Dest_Lat,Dest_Lon
0,Singapore Airlines,12,2023,4,4,4,4,4,9,1,...,False,False,False,False,False,True,-5.268581,142.097043,52.461441,9.687237
1,Singapore Airlines,2,2024,5,3,4,4,1,3,0,...,False,False,False,False,False,True,8.110132,98.305815,52.461441,9.687237
2,Singapore Airlines,2,2024,1,5,2,1,5,10,1,...,False,False,False,False,True,False,10.310328,123.979278,52.461441,9.687237
3,Singapore Airlines,2,2024,5,5,5,5,5,10,1,...,False,False,False,False,False,True,52.461441,9.687237,57.202226,-2.201736
4,Singapore Airlines,2,2024,5,5,5,5,5,10,1,...,False,False,False,False,True,False,52.461441,9.687237,48.688422,9.205393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5402,Korean Air,6,2016,5,4,5,3,4,7,1,...,False,False,True,False,False,False,-27.387621,153.121582,47.842976,106.766756
5403,Korean Air,6,2016,3,5,5,4,5,10,1,...,False,False,False,True,False,False,41.609391,41.600614,57.202226,-2.201736
5404,Korean Air,4,2016,4,5,5,5,1,2,0,...,False,False,True,False,False,False,34.284574,129.331353,47.842976,106.766756
5405,Korean Air,4,2016,5,1,3,4,5,10,1,...,False,False,True,False,False,False,47.842976,106.766756,-34.832022,-56.022260


In [9]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    
    a = sin(dlat / 2) * sin(dlat / 2) + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) * sin(dlon / 2)
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c  # Distance in kilometers
    return distance

# Calculate the distance between the two points
def calculate_duration(row):
    distance = haversine(row['Origin_Lat'], row['Origin_Lon'], row['Dest_Lat'], row['Dest_Lon'])

    # Assume an average speed of 800 km/h for the flight
    average_speed_kmph = 800

    # Calculate the flight duration
    flight_duration_hours = distance / average_speed_kmph

    # print(f"Flight duration: {flight_duration_hours:.2f} hours")
    return flight_duration_hours


In [ ]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    
    a = sin(dlat / 2) * sin(dlat / 2) + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) * sin(dlon / 2)
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c  # Distance in kilometers
    return distance

def calculate_duration(row):
    distance = haversine(row['Origin_Lat'], row['Origin_Lon'], row['Dest_Lat'], row['Dest_Lon'])

    average_speed_kmph = 800

    flight_duration_hours = distance / average_speed_kmph

    return flight_duration_hours

In [10]:
df['Duration'] = df.apply(calculate_duration, axis=1)
df.head()

,Airline,Month Flown,Year Flown,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended,...,Class_Premium Economy,Type of Traveller_Business,Type of Traveller_Couple Leisure,Type of Traveller_Family Leisure,Type of Traveller_Solo Leisure,Origin_Lat,Origin_Lon,Dest_Lat,Dest_Lon,Duration
0,Singapore Airlines,12,2023,4,4,4,4,4,9,1,...,False,False,False,False,True,-5.268581,142.097043,52.461441,9.687237,16.514702
1,Singapore Airlines,2,2024,5,3,4,4,1,3,0,...,False,False,False,False,True,8.110132,98.305815,52.461441,9.687237,11.500051
2,Singapore Airlines,2,2024,1,5,2,1,5,10,1,...,False,False,False,True,False,10.310328,123.979278,52.461441,9.687237,13.344673
3,Singapore Airlines,2,2024,5,5,5,5,5,10,1,...,False,False,False,False,True,52.461441,9.687237,57.202226,-2.201736,1.155124
4,Singapore Airlines,2,2024,5,5,5,5,5,10,1,...,False,False,False,True,False,52.461441,9.687237,48.688422,9.205393,0.526144


In [13]:
df.to_csv('static/data/airline_reviews_for_flask.csv')